# Гиперпицца

In [1]:
from html.parser import HTMLParser
from numpy import array as Arr
import pandas as pd
import os, io, re, random
import PySimpleGUI as sg

take_money_list, comp_cr2_dict = [], {}
emp_hr_count2_dict, make_deal_dict = {}, {}
emp_hr_total_dict, emp_cost_max = {}, {}
emp_hr_count3_dict, emp_hr_dict = {}, {}
comp_cr3_dict, comp_cr4_dict = {}, {}
emp_hr_count4_dict, buy_rnd4_list = {}, []


## Preprocess text
class MyHTMLParserHP(HTMLParser):
    def handle_data(self, data):
        if not re.match(r'^\s*$', data):
            result.append(data)

## Get table and result list
def table_result_score(filepath):
    global result
    table_new, result = [], []

    parser = MyHTMLParserHP()
    with io.open(filepath, encoding = 'utf-8') as file:
        for line in file:
            parser.feed(line)
    
    global player_name
    #player_name = [line for line in result if line.startswith('Игрок')][0][7:]
    score = float([line for line in result if line.startswith('Баланс')][0][8:])
    table = result[result.index('Таблица транзакций') + 3 : result.index('Описание транзакций')]
    table = [str(int(z)+1) if z.isdigit() else z for z in table]
    for i in range(0, len(table), 2):
        table_new.append((table[i], table[i + 1]))

    del result[:result.index('Описание транзакций') + 1]
    levels = [i for i, x in enumerate(result) if x[-2:] == ': ']
    #result = [result[i].lower() for i in range(len(result))]
    result = [result[i].lower() + result[i + 1].lower() if i in levels else result[i] for i in range(len(result))]
    result = [x[:-1]+str(int(x[-1])+1) if x.startswith('уровень игры:') else x for x in result]
    result = [result[i] for i in range(len(result)) if i - 1 not in levels]
    indexs = [i for i, x in enumerate(result) if x == 'выполнена' or x == 'отклонена']
    indexs.append(len(result))
    result = [result[indexs[n]:indexs[n + 1]] for n in range(len(indexs)-1)]
    result = [x if 'уровень игры: ' in ''.join(x) else \
            x.append('уровень игры: ' + table[table.index(x[1])+1]) or x for x in result]
    #result = [x[:x.index('отклонена')] if 'отклонена' in x else x for x in result]
    result = [x[:x.index('ожидает одобрения участников')] if 'ожидает одобрения участников' in x else x for x in result]
    return(table_new, result, score)

## Rules Function
def rules(code, filename, marks):
    numbers = Arr([0]*8)
    if code == "1.1":
        if ('зафиксировать инженерам распределение приза от корпорации X', '1') in tables[filename]:
            take_money_list.append(filename)
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "1.2":
        if ('подарить талеры', '1') in tables[filename] \
                and any([y[y.find('кто')+5:] in ' '.join(take_money_list).lower() \
                for y in sum([x for x in results[filename] if 'подарить талеры' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('кто')]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "1.3":
        if ('взять у скаута инструмент в аренду', '1') in tables[filename] \
                and any([y for y in sum([x for x in results[filename] if 'взять у скаута инструмент в аренду' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('cкаут')]): # english c
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "1.4":
        if ('покупка готовой продукции', '2') in tables[filename] or \
                any([y for y in sum([x for x in results[filename] if 'взять у скаута инструмент в аренду' in x \
                and 'уровень игры: 1' in x], []) if y.startswith('инженер')]):
            numbers += Arr([2,0,0,0,0,0,0,0])
    elif code == "1.5":
        if filename in ' '.join(take_money_list) \
                and all([('подарить талеры', k) not in tables[filename] for k in ['1', '2']]):
            numbers += Arr([0,0,1,1,0,0,0,0])
    elif code == "1.6":
        if filename in ' '.join(take_money_list) \
                and any([('подарить талеры', k) in tables[filename] for k in ['1', '2']]):
            numbers += Arr([0,0,0,0,0,1,0,0])
    elif code == "2.0":
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('название')]:
            comp_cr2_dict[comp_name].append(filename) if comp_name in comp_cr2_dict.keys() \
                    else comp_cr2_dict.update({comp_name: [filename]})
    elif code == "2.1":
        if all([(action, '2') in tables[filename] for action in \
                ['создать компанию', 'перевести талеры на счет компании', 'нанять сотрудника']]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "2.2":
        if ('перевести талеры на счет компании', '2') in tables[filename] \
                and max([len(x) for x in comp_cr2_dict.values() if filename in x]) > 1:
            numbers += Arr([0,0,1,0,0,1,0,0])
    elif code == "2.3":
        if any([y for y in sum([x for x in results[filename] if 'тип договора: продажа инструмента' in x \
                and 'уровень игры: 2' in x], [])]): # english c
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.4":
        if scores[filename] <= -400 and tables[filename].count(('нанять сотрудника', '2')) < 5:
            numbers += Arr([-1,0,-1,0,0,0,0,0])
    elif code == "2.5":
        if scores[filename] <= -400 and tables[filename].count(('нанять сотрудника', '2')) >= 5:
            numbers += Arr([-1,0,-1,0,0,1,0,0])
    elif code == "2.6":
        emp_hr_count2_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 2' in x], [])])
        emp_cost_max[filename[7:-5].lower()] = max([int(y[y.find('зарплата')+10:]) \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x and 'уровень игры: 2' in x \
                and any(['инженер: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('зарплата')], default=0)
        if emp_hr_count2_dict[filename] > 3:
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.7":
        if ('покупка готовой продукции', '2') in tables[filename] \
                and len([x for x in results[filename] if 'от компании получен  продукт(услуга): отдел закупок корпорации 1'
                in x or 'от компании получен  продукт(услуга): отдел закупок корпорации 2' in x]):
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "2.8":
        if any([filename in x for x in comp_cr2_dict.values()]) \
                and max([len(x) for x in comp_cr2_dict.values() if filename in x]) > 1 \
                and ('перевести талеры на счет компании', '2') not in tables[filename]:
            numbers += Arr([0,0,-2,0,0,0,1,0])
    elif code == "2.9":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 2' in x], []) if y.startswith('ceo')]):
            numbers += Arr([1,0,0,1,0,0,0,0])
        make_deal_dict[filename] = list(set([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 2' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')]))
    elif code == "3.0":
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('название')]:
            comp_cr3_dict[comp_name].append(filename) if comp_name in comp_cr3_dict.keys() \
                    else comp_cr3_dict.update({comp_name: [filename]})
    elif code == "3.1":
        if any([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('инвестор')]):
            numbers += Arr([0,0,0,2,0,0,0,0])
    elif code == "3.2":
        make_deal_x = ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 3' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if any([y not in make_deal_dict[filename]+['материалы'] for y in make_deal_x]) or make_deal_x.count('материалы') > 2:
            numbers += Arr([1,0,0,2,3,0,0,0])
    elif code == "3.3":
        if ('нанять сотрудника', '3') in tables[filename]:
            numbers += Arr([0,0,0,0,0,0,0,0])
    elif code == "3.4":
        emp_hr_count3_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 3' in x], [])])
        if emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,1,0,0,0,0,0])
    elif code == "3.5":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'отклонена' in x and 'уровень игры: 4' not in x and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,0,-1,0,0,0,0,0])
    elif code == "3.6":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'уровень игры: 4' not in x and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "3.7":
        if any([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and all(['от компании получен  продукт(услуга): компания х1' not in x, \
                         'от компании получен  продукт(услуга): компания х2' not in x]) \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]):
            numbers += Arr([1,1,1,0,0,0,0,0])
    elif code == "3.8":
        make_deal_dict[filename] += ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 3' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if len(make_deal_dict[filename]) != len(set(make_deal_dict[filename])):
            numbers += Arr([0,0,1,1,0,0,0,0])
        make_deal_dict[filename] = list(set(make_deal_dict[filename]))
    elif code == "3.9":
        emp_hr_total_dict[filename] = [y[y.find('инженер')+9:] \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and any(['ceo: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('инженер')]
        if sum([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('покупатель')]) >= 2:
            numbers += Arr([3,0,0,0,3,0,0,0])
    elif code == "3.10":
        if len([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('инвестор')]) > 1:
            numbers += Arr([0,0,1,0,1,0,0,0])
    elif code == "3.11":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and any(['от компании получен  продукт(услуга): компания х1' in x, \
                         'от компании получен  продукт(услуга): компания х2' in x]) \
                and 'уровень игры: 3' in x], []) if y.startswith('ceo')]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.12":
        emp_hr_dict[filename] = [y[y.find('инженер')+9:] \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x and 'уровень игры: 2' in x \
                and any(['ceo: '+filename[7:-5].lower() in z for z in x])], []) if y.startswith('инженер')]
        emp_hr_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x]) \
                and all(['инженер: '+emp not in x for emp in emp_hr_dict[filename]])]
        if any([int(_dict['зарплата']) > emp_cost_max[_dict['инженер']] for _dict in emp_hr_x]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.13":
        emp_hr_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x]) \
                and any(['инженер: '+emp in x for emp in emp_hr_dict[filename]])]
        if any([int(_dict['зарплата']) > emp_cost_max[_dict['инженер']] for _dict in emp_hr_x]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "3.14":
        revenue_x = [dict([xx.split(': ') for xx in x if ':' in xx]) for x in results[filename] if 'заключить сделки' in x \
            and 'уровень игры: 3' in x and any(['ceo: '+filename[7:-5].lower() in z for z in x])]
        if emp_hr_count3_dict[filename] > 3 and all([(float(x['цена'])*int(x['кол-во'])) < 10 for x in revenue_x]):
            numbers += Arr([0,0,0,0,0,3,0,0])
    elif code == "4.0":
        if any([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инвестор')]): #3.1 copy
            numbers += Arr([0,0,2,2,0,0,0,0])
        if len([y[y.find('инвестор')+10:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'инвестировать' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инвестор')]) > 1: #3.10 copy
            numbers += Arr([0,0,1,0,1,0,0,0])
        for comp_name in [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('название')]:
            comp_cr4_dict[comp_name].append(filename) if comp_name in comp_cr4_dict.keys() \
                    else comp_cr4_dict.update({comp_name: [filename]})
        for comp_name in [y[y.find('получатель компания')+21:] \
                for y in sum([x for x in results[filename] if 'покупка готовой продукции' in x \
                and 'уровень игры: 4' in x and 'от компании получен  продукт(услуга): отдел rnd корпорации х' in x], []) \
                if y.startswith('получатель компания')]:
            buy_rnd4_list.append(comp_name)
    elif code == "4.1":
        if ('закрыть компанию', '4') in tables[filename] and \
                max([len(x) for x in comp_cr4_dict.values() if filename in x], default=0) == 1:
            numbers += Arr([0,0,1,0,0,1,0,0])
    elif code == "4.2":
        make_deal_dict[filename] += ([y[y.find('тип договора')+14:] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x and 'уровень игры: 4' in x \
                and all([filename[7:-5].lower() not in z for z in x])], []) if y.startswith('тип договора')])
        if len(make_deal_dict[filename]) != len(set(make_deal_dict[filename])):
            numbers += Arr([0,0,1,1,0,0,0,0])
    elif code == "4.3":
        emp_hr_count4_dict[filename] = len([y[y.find('ceo')+5:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('ceo')]) - \
                len([y for y in sum([x for x in results[filename] if 'уволить сотрудника' in x \
                and 'уровень игры: 4' in x], [])])
        if emp_hr_count4_dict[filename] < min(3, emp_hr_count3_dict[filename]):
            numbers += Arr([0,0,1,0,0,0,0,0])
    elif code == "4.4":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'отклонена' in x and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,0,-1,0,0,0,0,0])
    elif code == "4.5":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr3_dict.items()) + list(comp_cr4_dict.items()) \
                          if filename in names]
        balances = []
        for comp in comp_x[:]:
            balance_x = 0
            index_max_x = max([results[filename].index(x) for x in results[filename] if 'нанять сотрудника' in x \
                            and 'компания: '+comp in x], default=0)
            for x in results[filename][:index_max_x+1]:
                if 'выполнена' in x and 'компани' in ' '.join(x) and comp in ' '.join(x):
                    if 'перевести талеры на счет компании' in x and 'собственник: '+filename[7:-5].lower() in x:
                        cost_raw = [y for y in x if y.startswith('взнос')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'нанять сотрудника' in x and 'ceo: '+filename[7:-5].lower():
                        cost_raw = [y for y in x if y.startswith('зарплата')][0]
                        balance_x -= int(cost_raw[cost_raw.find(' ')+1 :])
                    elif 'покупка готовой продукции' in x or 'заключить сделки' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        num_raw = [y for y in x if y.startswith('кол-во')][0]
                        if 'получатель компания: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                        elif 'ceo: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1:]) * int(num_raw[num_raw.find(' ')+1:])
                    elif 'купить продукт B2C' in x:
                        cost_raw = [y for y in x if y.startswith('цена')][0]
                        if 'у компании: '+comp in x:
                            balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                        elif 'покупатель: '+filename[7:-5].lower() in x:
                            balance_x -= int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
                    elif 'инвестировать' in x:
                        cost_raw = [y for y in x if y.startswith('инвестиции')][0]
                        balance_x += int(cost_raw[cost_raw.find(' ')+1 : cost_raw.rfind(' ')])
            balances.append(balance_x)
        if max(balances, default=0) >= 50 and emp_hr_count3_dict[filename] < min(3, emp_hr_count2_dict[filename]):
            numbers += Arr([0,1,0,0,0,0,0,0])
    elif code == "4.6":
        comp_x = [comp for comp, names in list(comp_cr2_dict.items()) + list(comp_cr4_dict.items()) if filename in names]
        if any([comp in buy_rnd4_list and buy_rnd4_list.count(comp) >1 for comp in comp_x]):
            numbers += Arr([1,0,1,0,0,0,0,0])
    elif code == "4.7":
        if any([y[y.find('от компании получен  продукт(услуга)')+38:]  in buy_rnd4_list \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 4' in x and 'ceo: {}'.format(filename[7:-5]).lower() not in x], []) \
                if y.startswith('от компании получен  продукт(услуга)')]):
            numbers += Arr([1,0,0,0,0,0,0,0])
    elif code == "4.8":
        if ('продать компанию', '4') in tables[filename]:
            numbers += Arr([2,2,1,2,2,0,0,0])
    elif code == "4.9":
        if ('продать компанию', '4') in tables[filename] \
                and ('создать компанию', '4') in tables[filename][tables[filename].index(('продать компанию', '4')):]:
            numbers += Arr([1,0,0,0,1,0,0,0])
    elif code == "4.10":
        if ('продать компанию', '4') in tables[filename] \
                and any([y[y.find('инженер')+9:] in filename.lower() \
                for y in sum([x for x in results[filename] if 'нанять сотрудника' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('инженер')]):
            numbers += Arr([-1,-1,-1,-1,-1,0,0,0])
    elif code == "4.11":
        comp_cr_total_x = [y[y.find('название')+10:] \
                for y in sum([x for x in results[filename] if 'создать компанию' in x], []) if y.startswith('название')]
        if len(comp_cr_total_x) > 1:
            if all([y[y.find(':')+2:] in comp_cr_total_x \
                   for y in sum([x for x in results[filename] if 'заключить сделки' in x], []) \
                   if any([y.startswith('от компании'), y.startswith('получатель')])]):
                numbers += Arr([0,0,0,0,0,0,4,0])
    elif code == "4.12":
        if any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 3' in x], []) if y.startswith('покупатель')]) \
                and any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('покупатель')]):
            numbers += Arr([0,1,0,1,0,0,0,0])
    elif code == "4.13":
        if any([y[y.find('ceo')+5:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'заключить сделки' in x \
                and 'уровень игры: 4' not in x], []) if y.startswith('ceo')]) \
                and any([y[y.find('покупатель')+12:] not in [filename[7:-5].lower()]+emp_hr_total_dict[filename] \
                for y in sum([x for x in results[filename] if 'купить продукт B2C' in x \
                and 'уровень игры: 4' in x], []) if y.startswith('покупатель')]):
            numbers += Arr([0,0,0,1,1,0,0,0])
    elif code == "4.14":
        if any([y[y.find('ceo')+5:] not in filename.lower() \
                for y in sum([x for x in results[filename] if 'покупка готовой продукции' in x \
                and 'уровень игры: 4' not in x], []) if y.startswith('ceo')]):
            numbers += Arr([0,1,0,0,0,0,0,0])
            
    if filename in marks.keys():
        marks[filename] += numbers
    else:
        marks.update({filename: Arr([0]*8)})
        
    for key, value in marks.items():
        marks[key] = [min(y, 10) for y in value]
    
    return(1 if list(numbers) != [0]*8 else 0)

In [2]:
#os.chdir('p:///data')
#os.getcwd()

def main(path):
    os.chdir(path)
    global tables, results, scores
    tables, results, scores = {}, {}, {}
    all_values, marks = {}, {}
    files = [filename for filename in os.listdir() if filename.startswith('_игрок') and filename.endswith('.html')]
    #for n in [1.1, 2.0, 2.6, 2.9, 3.4, 3.8, 3.9, 3.12, 4.0, 4.2]: #[f"1.{i}" for i in range(1,7)] +
    codes = [f"2.{i}" for i in range(10)] + [f"3.{i}" for i in range(15)] \
            + [f"4.{i}" for i in range(15)]
    for filename in files:
        try:
            tables[filename], results[filename], scores[filename] = table_result_score(filename)
        except:
            pass
    for code in codes:
        all_values[code] = []
        for filename in files:
            try:
                all_values[code].append(rules(code, filename, marks))
            except Exception as e:
                #print(e)
                all_values[code].append(0)
                marks[filename] = [0]*8
                pass

    final_table = pd.DataFrame(all_values, index=[filename[7:-5] for filename in files])
    final_table['score'] = [list(x)[::-1][3:]+(list(x)[::-1][:3]) for x in marks.values()]
    
    return(final_table)

# PizzaChart

In [3]:
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.projections import register_projection
from matplotlib.projections.polar import PolarAxes
from matplotlib.transforms import Affine2D
from matplotlib.ticker import FixedLocator
from matplotlib.spines import Spine
from matplotlib.path import Path
from math import pi

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import PySimpleGUI as sg

In [4]:
def radar_factory(num_vars, frame='circle'):
    
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    
    class RadarAxes(PolarAxes):
        name = 'radar'

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            if x[0] != x[-1]:
                x = np.concatenate((x, [x[0]]))
                y = np.concatenate((y, [y[0]]))
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return Circle((0.5, 0.5), 0.5)
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

        def draw(self, renderer):
            if frame == 'polygon':
                gridlines = self.yaxis.get_gridlines()
                for gl in gridlines:
                    gl.get_path()._interpolation_steps = num_vars
            super().draw(renderer)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                spine.set_transform(Affine2D().rotate(pi/8).scale(0.5).translate(.5, .5)
                                    + self.transAxes)

                return {'polar': spine}
            else:
                raise ValueError("unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

In [5]:
def web1(filename, col_names, numbers):
    dict1 = {col: 0 for col in col_names}
    N = 8
    theta = radar_factory(8, frame='polygon')

    angles = np.linspace(0, 2 * pi, N, endpoint=False)
    angles_mids = angles + (angles[1] / 2)

    fig, ax = plt.subplots(figsize=(11, 8), subplot_kw=dict(projection='radar'))
    ax.xaxis.set_minor_locator(FixedLocator(angles))
    fig.subplots_adjust(top=0.95, bottom=0.05, left=0.07, right=0.85)
    plt.ylim([0, 10])

    ax.set_theta_offset(pi/8-pi/4)
    ax.set_theta_direction(1)
    ax.set_xticks(angles_mids)
    ax.set_xticklabels(dict1.keys())
    ax.grid(True, axis='x', which='minor')
    ax.grid(False, axis='x', which='major')
    ax.grid(True, axis='y', which='major')
    ax.set_rgrids([2,4,6,8,10], [])
    #ax.set_title(file_name,  position=(0.5, 1.1), ha='center')
    color1='springgreen'
    color2='seagreen'
    color3='tomato'
    color4='red'
    for i in range(6):
        ax.fill_between(angles[i:i+2], 0, numbers[i] , facecolor=color1, edgecolor=color2, linewidth=2)
    ax.fill_between(angles[5:7], 0, numbers[5], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between(angles[6:8], 0, numbers[6], facecolor=color3, edgecolor=color4, linewidth=2)
    ax.fill_between([angles[7],(angles[7]+angles[1])], 0, numbers[7], facecolor=color3, edgecolor=color4, linewidth=2)

    for index, label in enumerate(ax.get_xticklabels()):
        label.set_weight("bold")
        label.set_fontsize(14)
        if index in [0, 1, 7]:
            label.set_horizontalalignment("left")
        elif index in [3, 4, 5]:
            label.set_horizontalalignment("right")

    plt.savefig(filename, transparent=True)
    return(filename)

In [6]:
col1 = 'Дисконтирование\nустаревших\nрешений'
col2 = 'Давление на\nнезависимого\nпартнера'
col3 = 'Конструирование\n компаний'
col4 = 'Препарирование\nтехнологий'
col5 = 'Скрининг\nшансов'
col6 = 'Борьба за\nпривилегии'
col7 = 'Фиктивная\nдеятельность'
col8 = 'Отношения\nвместо дела'
col_names = [col1, col2, col3, col4, col5, col6, col7, col8]

# Мегаотчет

In [7]:
from docxtpl import DocxTemplate, InlineImage, RichText
from html.parser import HTMLParser
from bs4 import BeautifulSoup as BS
from docx.shared import Mm, Cm
import PySimpleGUI as sg
import pandas as pd
import datetime
import os
import io
import re
import regex
import docx

pd.options.mode.chained_assignment = None

file_path = "_сквозной_список_все_личные_счета.html"

blacklist = ['Глеб Черепанов', 'Черепанов Глеб', 'Андрей Богдашёв', 'Богдашёв Андрей', 'Андрей Богдашев', 'Богдашев Андрей',\
        ' Михаил Ковалёв', 'Ковалёв Михаил', 'Михаил Ковалев', 'Ковалев Михаил', 'Столярова Полина', 'Полина Столярова',\
        'Кристина Клюева', 'Клюева Кристина', 'Светлана Андреева', 'Андреева Светлана', 'Владислав Богдан', 'Богдан Владислав',\
        'Богдан Влад', 'Латышев Николай', 'Червоная Ольга', 'Александр Кожевников', 'Попов Дмитрий',\
        'Бабенков Виктор', 'Щербич Данила', 'Щербич Данил', 'Тимофеев Андрей', 'Параев Павел', 'Ильина Наташа',\
        'Червоная Ольга', 'Червонная Ольга', 'Червоная Оля', 'Андреева Светлана', 'Андреева Света', 'Клюева Кристина',\
        'Сыромятников Максим', 'Саруханова Диана', 'Лагнер Александра', 'Лагнер Саша', 'Маяренков Сергей', 'Кучеренко Максим',\
        'Карелина Юлия', 'Логов Александр', 'Директор НИИ'
]

In [8]:
def window_string(key, text_size, input_size, tooltip=None):
    return ([sg.Text(key, size=(text_size,1)), sg.InputText(key=key, size=(input_size,1), tooltip=tooltip)])

def make_window():        
    layout = [
        [sg.HorizontalSeparator()],
        [sg.Text('Папка с данными игры', size=(20,1)),  sg.FolderBrowse('Путь', size=(4,1), target='folder_path'), sg.InputText(size=(30, 1), key='folder_path')],
        [sg.Text('Асессор комментарии', size=(20,1)),  sg.FileBrowse('Путь', size=(4,1), target='table_final'), sg.InputText(size=(30, 1), key='table_final')],
        [sg.ProgressBar(100, orientation='h', size=(33,20), key='progress_bar')],
        [sg.HorizontalSeparator()],
        [sg.Output(size=(56,14))],
        [sg.Submit('Запустить'), sg.Cancel('Выйти')],
        [sg.HorizontalSeparator()],
        ]
    
    return sg.Window('Мегаотчет v0.7_l3', layout)

In [9]:
def html_read(result_list):
    player_name = result_list[1][7:] if result_list[1][-1] != ' ' else result_list[1][7:-1]
    table = result_list[result_list.index("Таблица транзакций") + 3 : result_list.index("Описание транзакций")]
    table = {table[i]: table.count(table[i]) for i in range(0, len(table), 2)}
    del result_list[:result_list.index("Описание транзакций") + 1]
    levels = [i for i, x in enumerate(result_list) if x[-2:] == ": "]
    result_list = [result_list[i].lower() + result_list[i + 1].lower() if i in levels else result_list[i] for i in range(len(result_list))]
    result_list = [result_list[i] for i in range(len(result_list)) if i - 1 not in levels]
    indexs = [i for i, x in enumerate(result_list) if x == "выполнена"]
    indexs.append(len(result_list))
    result_list = [result_list[indexs[n]:indexs[n + 1]] for n in range(len(indexs)-1)]
    result_list = [x if "уровень игры: " in ''.join(x) else x.append("уровень игры: " + str(table[x[1]])) or x for x in result_list]
    return (player_name, table, result_list)

def table_load():
    file_path_table = folder_path + file_path
    table = pd.read_html(file_path_table,encoding = 'utf-8')[0]
    table = table[table['Фамилия'].apply(lambda x: not x.startswith('Бот') and not x.startswith('Аватар'))]
    table['Баланс'] = table['Баланс'].apply(int)
    table['ФИО'] = table['Фамилия']+' '+table['Имя']+' '+table['Отчество']
    table = table[table['ФИО'].apply(lambda x: all([y not in x for y in blacklist]))]
    table = table[table['Роль'].apply(lambda x: not x.startswith('Директор Фонда'))]
    table = table.reset_index()
    if 'index' in table.columns:
        table.drop('index', axis=1, inplace=True)
    return(table)
    
def counter(values):
    class MyHTMLParser(HTMLParser):
        def handle_data(self, data):
            if not re.match(r'^\s*$', data):
                result_list.append(data)

    comps, top_players_count, trans_dict, trans_dict_deal = len(pd.read_html(comp_path, encoding='utf-8')[-1])-2, 0, {}, {}
    trans_all, trans_fire, trans_switch, trans_contr, trans_create, trans_close, trans_money, trans_deal, players_active = 0, 0, 0, 0, 0, 0, 0, 0, []
    comp_closed, comp_sold, comp_invested = set(), set(), set()
    for file_name in os.listdir(folder_path)[:]:
        if file_name.startswith("_игрок") and file_name.endswith(".html") and 'Бот' not in file_name and 'Аватар' not in file_name:
            result_list = []
            parser = MyHTMLParser()
            with io.open(folder_path+file_name, encoding = 'utf-8') as file:
                for line in file:
                    parser.feed(line)
            player_name, result_table, result_list = html_read(result_list)
            
            trans_all = len(result_list)
            if 'автоматическое увольнение после окончания уровня' in result_table:
                trans_fire = result_table['автоматическое увольнение после окончания уровня']
            if 'cмена роли' in result_table:
                trans_switch = result_table['cмена роли']
            trans_contr = len([x for x in result_list if 'создать новый контракт' in ''.join(x)])
            trans_create = len([x for x in result_list if 'создать компанию' in ''.join(x)])
            trans_close = len([x for x in result_list if 'закрыть компанию' in ''.join(x)])
            trans_money = len([x for x in result_list if 'перевести талеры на счет компании' in ''.join(x)])
            trans_deal = int(len([x for x in result_list if 'заключить сделки' in ''.join(x)])/2)
            trans_right = trans_create+trans_close+trans_money+int((trans_all-trans_fire-trans_switch-trans_contr)/2)
            trans_dict.update({player_name: trans_right})
            trans_dict_deal.update({player_name: trans_deal})
            
            levels = set([[y[y.find(':')+2:] for y in x if y.startswith('уровень')][0] for x in result_list])
            if len(levels) > 1 and max(levels, key=int) == '3':
                players_active.append(player_name.replace('  ', ' '))
                
            try:
                comp_closed.update([[y[y.find(':')+2:] for y in x if y.startswith('название')][0] for x in result_list if 'закрыть компанию' in ''.join(x)])            
                comp_sold.update([[y[y.find(':')+2:] for y in x if y.startswith('компания продавец')][0] for x in result_list if 'продать компанию' in ''.join(x)])
                comp_invested.update([[y[y.find(':')+2:] for y in x if y.startswith('в компании')][0] for x in result_list if 'инвестировать' in ''.join(x)])
            except:
                print('ошибка в: '+file_name)
            #print(trans_create, trans_close, trans_money, trans_all, trans_fire, trans_switch, trans_contr, trans_dict[player_name])
    top_player_list = []
    for file_, value_ in tables.items():
        if ('заключить сделки', '4') in value_ and file_[7:-5] in list(table[table['Роль'] == 'Предприниматель']['ФИО']):
            top_players_count += 1
    top_players_part = round(100*top_players_count/len(table))
    top_players_level = 'высокий' if top_players_part >= 0.12 else 'средний'
    eng_active = len(table[(table['Роль']=='Инженер') & (table['ФИО'].isin(players_active))])
    ent_active = len(table[(table['Роль']=='Предприниматель') & (table['ФИО'].isin(players_active))])
    
    trans_right = sum(trans_dict.values())
    trans_deal = sum(trans_dict_deal.values())
    
    for key, value in zip(['top_players_part', 'top_players_level', 'comps', 'eng_active', 'ent_active', 'trans_right', 'trans_deal', 'comp_closed', 'comp_sold', 'comp_invested'],
                          [top_players_part, top_players_level, comps, eng_active, ent_active, trans_right, trans_deal, len(comp_closed), len(comp_sold), len(comp_invested)]):
        values[key] = value
    table_load()
    
    trans_deal_2 = 0
    for table_ in tables.values():
        trans_deal_2 += table_.count(('заключить сделки', '2'))
    values['trans_deal_2'] = int(trans_deal_2 / 2)
    
    return(trans_dict)

def search(names, name):
    try:
        name1, name2 = name.split()
        result = [name for name in names if name1 in name.split(' ') and name2 in name.split(' ')]
        if result:
            return max(result)
        else:
            return(name)
    except:
        return(name)

def round_except(x):
    try:
        return round(x)
    except:
        return x
    
def counter_new(table):
    values['conclusion'] = ''
    
    values['last_level'] = '4'
    players_last_level_active = 0
    status_final_dict = {}
    for file_name in os.listdir(folder_path)[:]:
        if file_name.startswith("_игрок") and file_name.endswith(".html") and all([name not in file_name for name in ['Бот', 'Аватар', 'Акционер']]):
            status_final_dict[file_name] = 'Выбыший'
            transactions = pd.read_html(folder_path+file_name,encoding = 'utf-8')[0]
            if int(values['last_level'])-1 in set(transactions['Уровень']):
                status_final_dict[file_name] = 'Финалист'
                players_last_level_active += 1
    for index, name in enumerate(zip(table['Фамилия'], table['Имя'], table['Отчество'])):
        table.at[index, 'Файл'] = [file_name for file_name in os.listdir(folder_path) if all([x in file_name for x in name]) and '.png' not in file_name][0]
    table['Статус'] = table['Файл'].apply(lambda x: status_final_dict[x])
    table_final = pd.read_excel(values['table_final'])
    table_final['Фамилия'] = table_final['Фамилия'].str.replace('\xa0', '')
    table_final['Имя'] = table_final['Имя'].str.replace('\xa0', '')
    table_final['Отчество'] = table_final['Отчество'].str.replace('\xa0', '')
    table_final.sort_values('Эшелон', inplace=True)
    table_final['№'] = range(1,len(table_final)+1)
    table_final['ФИО'] = table_final['Фамилия']+' '+table_final['Имя']+' '+table_final['Отчество']
    table_1 = pd.merge(table[table.columns.difference(['Имя', 'Фамилия', 'Отчество'])],\
                       table_final[table_final.columns.difference(['ФИО'])], left_on="Контакт", right_on="Email", how="right")
    table_2 = pd.merge(table[table.columns.difference(['Имя', 'Фамилия', 'Отчество'])], table_final, left_on="ФИО", right_on="ФИО", how="right")
    table_1.update(table_2.iloc[table_1[table_1['Контакт'].isnull()].index])
    table_final = table_1
    table = table[~table['Контакт'].isin(table_final['Email'])]
    table = table[~table['ФИО'].isin(table_final['ФИО'])]
    table['Эшелон'] = table['Статус'].apply(lambda x: 3 if x == 'Финалист' else 4)
    table_final = pd.concat([table_final, table[table['Эшелон'] == 3], table[table['Эшелон'] == 4]])
    table_final.reset_index(drop=True, inplace=True)
    table_final['№'] = table_final.index + 1
    table_final.fillna('?', inplace=True)
    table_final['Эшелон'] = table_final['Эшелон'].apply(round_except)
    table_final['Баланс'] = table_final['Баланс'].apply(lambda x: int(x) if isinstance(x, float) else x)
    try:
        table_final['Телефон'] = table_final['Телефон'].apply(lambda x: int(x) if isinstance(x, float) else x)
        table_final['Курс'] = table_final['Курс'].apply(lambda x: int(x) if isinstance(x, float) else x)
    except:
        pass
    table_final['Почта'] = table_final['Контакт']
    table_final[table.columns[5]] = table_final[table.columns[5]].apply(round_except)
    table_final[table.columns[6]] = table_final[table.columns[6]].apply(round_except)
    values['players_all'] = len(table_final)
    values['players_last_level_active'] = min(players_last_level_active, values['players_all'])
    
    col_labels = ['№', 'Фамилия', 'Имя', 'Отчество', table.columns[5], table.columns[6], 'Контакт', 'Телефон', 'Роль', 'Баланс', 'Статус', 'Файл']
    #col_labels_final = ['№', 'Фамилия', 'Имя', 'Отчество', 'Контакт', 'Телефон', 'Роль', 'Баланс', 'Статус']
    col_labels_final = ['№', 'ФИО', 'Почта', 'Телефон', 'Роль', 'Баланс'] #, 'Статус'
    values['key'] = 2
    values['col_labels_final'] = col_labels_final
    col_labels = col_labels + ['Комментарий', 'Рекомендация для участника', 'Эшелон', 'Эшелон_класс']
    
    doc = DocxTemplate("Ввод_new/тело_отчёта_шаблон_{}_с_заголовком.docx".format(values['key']))
    ###doc = DocxTemplate("Ввод_new/статистика_в_тренингах_шаблон_new.docx")
    for key in list(values.keys()):
        if isinstance(key, str) and isinstance(values[key], str):
            key_new = key.replace(" ", "_")
            values[key] = values[key].replace("Нет", "")
            values[key_new] = values.pop(key)
    values['map_1'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level2.png', Cm(10))
    values['map_2'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level3.png', Cm(10))
    values['map_3'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level4.png', Cm(10))
    
    echelon_1, echelon_2, echelon_3, echelon_4 = [], [], [], []
    echelon_dict = {1: echelon_1, 2: echelon_2, 3: echelon_3, 4: echelon_4}
    
    table_final['Эшелон_класс'] = table_final.apply(lambda x: echelon_classify(x['Файл'], x['Роль'], x['Эшелон'], x['Баланс']), axis=1).values
    for rating, name_1, name, name_2, col_1, col_2, email, phone, role, score, status_final, file_name, comment, recomm, echelon, echelon_class in table_final[col_labels].values[:]:
        try:
            name_1 = str(name_1)
            name = str(name)
            name_2 = str(name_2)
            phone = int(phone)
        except:
            pass
        #for file_name in os.listdir(folder_path):
         #   if all([n in file_name for n in [name_1, name, name_2]]) and file_name.endswith(".html"):
        status = '-'
        transactions = pd.read_html(folder_path + file_name)[0]
        if 'продать компанию' in transactions['Транзакция'].values:
            status = 'Продал компанию'
        elif 'инвестировать' in transactions['Транзакция'].values:
            status = 'Получил инвестиции'
        else:
            try:
                max_level = str(max(set(transactions['Уровень'])))
            except:
                max_level = '0'
            if int(max_level)+1 == int(values['last_level']):
                status = 'Дошёл до конца игры'
            else:
                status = 'Выбыл на {} уровне'.format(max_level)

        if pd.isna(email):
            email = '-'   

        #doc_disk = DocxTemplate("Ввод_new/Профиль {}.{} эшелон - диск.docx".format(echelon, values['key']))
        with open(folder_path + file_name, 'r', encoding='utf-8') as f:
            html = f.read()
        soup = BS(html, features='lxml')
        for image_link in soup.find_all('img'):
            if image_link['src'][:9] == 'generated':
                map_path = folder_path + image_link['src'][:image_link['src'].find('?')]
                if os.path.getsize(map_path) > 500:
                    player_map = InlineImage(doc, map_path, Cm(10))
                elif len(tables[file_name]) > 0:
                    player_map = 'Пропуск'
                else:
                    player_map = 'Участник не совершил достаточно экономических действий для формирования графа деловых связей'
                #player_map_disk = InlineImage(doc_disk, folder_path + image_link['src'][:image_link['src'].find('?')], Cm(10))
        web1(folder_path+file_name+'.png', col_names, final_table.loc[file_name[7:-5]]['score'])
        pizza = InlineImage(doc, folder_path+file_name+'.png', Cm(10))
        #pizza_disk = InlineImage(doc_disk, folder_path+file_name+'.png', Cm(10))
        item = {'name': name_1+' '+name+' '+name_2, 'email': email, 'date': values['game_date'], 'role': role, 'score': score, 'rating': rating, 'status': status,
                'transactions': len(transactions), 'comment': comment, 'recomm': recomm, 'phone': phone, 'col_1': col_1, 'col_2': col_2, 'key': values['key']}
        item.update(key_actions(file_name))
        #doc_disk.render({'item': item})
        #doc_disk.save(f'Вывод/{date_time}/эшелон '+str(echelon)+'/'+item['name']+'.docx')
        item['pizza'] = pizza
        item['map'] = player_map
        if echelon in [3, 4]:
            item['comment'] = echelon_comment(echelon_class)
        echelon_dict[echelon].append(item)
        window['progress_bar'].update(round(100*rating/len(table_final)))
    values['table_final'] = table_final[values['col_labels_final']].values
    values['table_1'] = table_final[table_final['Эшелон'] == 1][values['col_labels_final']].values
    values['table_2'] = table_final[table_final['Эшелон'] == 2][values['col_labels_final']].values
    values['table_3'] = table_final[table_final['Эшелон'] == 3][values['col_labels_final']].values
    values['table_4'] = table_final[table_final['Эшелон'] == 4][values['col_labels_final']].values
    values['echelon_1'], values['echelon_2'], values['echelon_3'], values['echelon_4'] = echelon_1, echelon_2, echelon_3, echelon_4
    values['echelon_1_len'], values['echelon_2_len'], values['echelon_3_len'], values['echelon_4_len'] = len(echelon_1), len(echelon_2), len(echelon_3), len(echelon_4)
    
    doc.render(values)
    doc.save(filepath)
    
    return(table_final)

def key_actions(filename):
    df_t = pd.DataFrame()
    key_actions = {'k1': '', 'k2': '', 'k3': '', 'k4': '', 'k5': ''}
    """
    if final_table.at[filename[7:-5], '1.3']:
        key_actions['k1'] = 'Проинвестировал в повышение производительности инженерного труда'
    elif final_table.at[filename[7:-5], '1.1']:
        key_actions['k1'] = 'Получил приз по результатам конкурса инженерных решений'
    elif final_table.at[filename[7:-5], '1.4']:
        key_actions['k1'] = 'Успешно идентифицировал возможность заработка в качестве сотрудника корпорации'
        """
    if filename in sum(comp_cr2_dict.values(), []):
        key_actions['k1'] = 'При первой возможности перешёл в предпринимательскую позицию'
    else:
        key_actions['k1'] = 'Пропуск'

    if final_table.at[filename[7:-5], '2.7']:
        key_actions['k2'] = 'Прошёл квалификацию на роль поставщика крупной корпорации'
    elif tables[filename].count(('заключить сделки', '3')) >2: #and not final_table.at[filename[7:-5], '1.4']:
        key_actions['k2'] = 'Смог определить рамки и сконструировать деятельность компании' \
        'в условиях высокой неопределённости и отсутствия внешнего заказчика'
    elif filename in sum(comp_cr2_dict.values(), []):
        key_actions['k2'] = 'При первой возможности перешёл в предпринимательскую позицию'
    #elif final_table.at[filename[7:-5], '1.4']:
    #    key_actions['k2'] = 'Успешно освоил рамки поставленной старшим партнёром позиции'
    else:
        key_actions['k2'] = 'Пропуск'

#and not final_table.at[filename[7:-5], '1.4']
    if tables[filename].count(('заключить сделки', '3')) >2 \
            and key_actions['k2'] != 'Смог определить рамки и сконструировать деятельность компании' \
            'в условиях высокой неопределённости и отсутствия внешнего заказчика':
        key_actions['k3'] = 'Смог определить рамки и сконструировать деятельность компании' \
        'в условиях высокой неопределённости и отсутствия внешнего заказчика'
    elif any([y[y.find('инвестор')+10:] not in filename.lower() \
            for y in sum([x for x in results[filename] if 'инвестировать' in x], []) if y.startswith('инвестор')]):
        key_actions['k3'] = 'Успешно провёл переговоры с инвестором и получил инвестиции' #final_table.at[filename[7:-5], '1.4'] and
    elif len([x for x in results[filename] if 'заключить сделки' in x \
            and 'тип договора: продажа инструмента' not in x and 'уровень игры: 3' in x]) >1:
        key_actions['k3'] = 'Выполнил задание старшего партнёра на строительство производства'
    else:
        key_actions['k3'] = 'Пропуск'

    if final_table.at[filename[7:-5], '4.14']:
        key_actions['k4'] = 'Организовал производство крупной партии продукции по заказу старшего партнёра'
    elif final_table.at[filename[7:-5], '4.7']:
        key_actions['k4'] = 'Корректно идентифицировал и занял нишу в системе разделения труда'
    elif any([y[y.find('инвестор')+10:] not in filename.lower() \
            for y in sum([x for x in results[filename] if 'инвестировать' in x], []) if y.startswith('инвестор')]) \
            and key_actions['k3'] != 'Успешно провёл переговоры с инвестором и получил инвестиции':
        key_actions['k4'] = 'Успешно провёл переговоры с инвестором и получил инвестиции'
    elif final_table.at[filename[7:-5], '4.8']:
        key_actions['k4'] = 'Продал построенную компанию'
    else:
        key_actions['k4'] = 'Пропуск'

    if final_table.at[filename[7:-5], '4.8'] and key_actions['k4']!= 'Продал построенную компанию':
        key_actions['k5'] ='Продал построенную компанию'
    elif ('найм в качестве директора компании', '4') in tables[filename] and ('продать компанию', '4') in tables[filename] \
            and ('заключить сделки', '4') in tables[filename][tables[filename].index(('продать компанию', '4')):]:
        key_actions['k5'] ='После продажи компании принял предложение о работе наёмным директором \
                и подтвердил эффективность проданной компании'
    elif final_table.at[filename[7:-5], '4.9']:
        key_actions['k5'] ='После продажи компании открыл новую компанию'
    else:
        key_actions['k5'] ='Пропуск'

    return(key_actions)

def echelon_classify(filename, role, echelon, score):
    if int(echelon) == 3:
        if role in ['Предприниматель', 'Директор X']:
            if tables[filename].count(('заключить сделки', '4')) >0:
                return('Активный предприниматель')
            else:
                return('Неактивный предприниматель')
        else:
            employers = set([y[y.find('ceo')+5:] for y in \
                    sum([x for x in results[filename] if 'нанять сотрудника' in x], []) if y.startswith('ceo')])
            if score >= 120 or len(employers) >= 2:
                return('Активный инженер')
            else:
                return('Обычный инженер')
    elif int(echelon) == 4:
        levels = [int(record[1]) for record in tables[filename]]
        if levels != []:
            if role == 'Предприниматель':
                if max(levels) == 3:
                    return('Предприниматель 2')
                else:
                    return('Предприниматель 1')
            else:
                if max(levels) == 3:
                    return('Инженер 2')
                elif max(levels) == 2:
                    return('Инженер 1')
                else:
                    return('Инженер 0')
        else:
            return('Инженер 0')
    else:
        return(echelon)
    
def echelon_comment(e_class):
    if e_class in comm_dict.keys():
        comm_list = comm_dict[e_class]
        comm_dict[e_class].append(comm_list[0])
        comm_dict[e_class].pop(0)
        return(comm_dict[e_class][-1])
    else:
        return(e_class)

comm_dict = {'Активный предприниматель':
                ['Участник проявил мотивацию к предпринимательской деятельности и значительный уровень резильентности, однако не продемонстрировал достаточного уровня предпринимательских компетенций для того, чтобы быть рекомендованным к включению в предпринимательские проекты без инвестиций в дальнейшую подготовку. Для выхода на уровень, необходимый для участия в предпринимательских проектах, участник нуждается как в освоении предметных знаний об экономических и юридических аспектах деятельности технологической компании, так и в изучении базовых принципов управления производством.',
                 'Участник показал себя активным и упорным предпринимателем, однако на данный момент не обладает набором базовых предпринимательских компетенций, достаточным для того, чтобы быть включённым в реальную предпринимательскую деятельность без высоких рисков. При этом участник  годен к дальнейшему обучению - освоение предметных знаний об экономике стартапов и принципах работы с инвестиционными деньгами позволит участнику компенсировать существующий дефицит компетенций и потенциально выйти на уровень, позволяющий рассматривать его как кандидата на включение в предпринимательские проекты.',
                 'Участник продемонстрировал высокий уровень активности и плотности деятельности, что может указывать как на высокий уровень исполнительности, так и на наличие внутренней мотивации к предпринимательской деятельности. Однако участник не продемонстрировал уровень базовых предпринимательских компетенций, ожидаемый от человека в позиции технологического предпринимателя - на данном уровне подготовки участник не может быть рекомендован к включению в предпринимательские проекты. При условии прохождения подготовки в таких областях как: управление компанией, анализ рынка и работа с инвестиционными деньгами, участник может потенциально рассматриваться как кандидат на включение в предпринимательские проекты.'
                ],
            'Неактивный предприниматель':
                ['Участник показал себя резильентным и исполнительным администратором, не обладающим, однако значительной внутренней мотивацией к деятельности технологического предпринимателя. Участник может ответственно выполнять порученную работу в новой среде и тяжёлых внешних условиях, однако только на позиции, получающей указания "сверху" - к самостоятельному конструированию деятельности участник не способен. Участника можно потенциально рассматривать как кандидата на любые позиции, требующие ответственности и стрессоустойчивости, но следует понимать, что он не обладает высоким уровнем предметных экономических знаний и управленческих компетенций.',
                 'Участник продемонстрировал способность вести сложную деятельность в стрессовой ситуации, однако не продемонстрировал достаточного уровня мотивации или компетенций, чтобы рассматриваться как кандидат на переход в предпринимательскую позицию. Ему, тем не менее, может быть доверен широкий спектр управленческих задач, не требующих высокого уровня погруженности в предмет экономики.',
                 'Уровень предпринимательских компетенций и внутренней мотивации к предпринимательской деятельности, продемонстрированные участником, недостаточны для того, чтобы рекомендовать его к включению в предпринимательские проекты. Однако, участник проявил значительный уровень ответственности и исполнительности в ведении деятельности, к которой не имел внутренней мотивации. Это делает его потенциальным кандидатом на занятие управленческих позиций, требующих резильентности и ответственного подхода к деятельности, и не предполагающих работы с такими экономическими предметами как организация производства, анализ рынка или конструирование новых форм деятельности.'
                ],
            'Активный инженер':
                ['Участник не проявил значительного интереса к предпринимательской позиции и предпринимательскому типу действия, но очень хорошо проявил себя на не-управленческих исполнительских позициях. Рациональный, ответственный и стрессоустойчивый участник, прекрасно понимающий рамки позиции исполнителя и чётко соблюдающий их. Вероятнее всего склонен только к наёмному труду в своей избранной предметной сфере, но имеет все необходимые характеристики, чтобы быть надёжным и эффективным исполнителем в рамках подобной деятельности.',
                 'Хороший сотрудник - не имеет значительных мотивации или интереса к предпринимательской деятельности, но полностью отвечает требованиям к предметному исполнителю: стрессоустойчив, целеустремлён и хорошо понимает рамки своей позиции и не нарушает их. К включению в предпринимательские проекты может быть рекомендован только на позициях инженера или специалиста.',
                 'Участник показал себя отличным исполнителем без какого-либо интереса к предпринимательской деятельности. Ответственен, стрессоустойчив, обладает хорошим пониманием "исполнительской рациональности" - хорошо понимает рамки договорённостей с работодателем, следует им. Продемонстрировал важное умение читать и понимать технические задания. Может быть рекомендован к включению в предпринимательские проекты на позициях инженера или предметного специалиста.'
                ],
            'Обычный инженер':
                ['Участник не продемонстрировал значительного интереса к предпринимательской деятельности, но показал себя как ответственный и стрессоустойчивый исполнитель. В деятельности не проявил инициативы или ярко выраженных компетенций, но упорно и устойчиво вёл деятельность в сложной ситуации. Как исполнитель не имеет ярко выраженных недостатков - может быть рекомендован к включению в широкий спектр проектов на исполнительских позициях.',
                 'Резильентный и стрессоустойчивый участник, не продемонстрировавший мотивации или даже интереса к ведению предпринимательской деятельности. Показал позитивные результаты в позиции исполнителя - ответственный и устойчивый, способен к работе в стрессовых ситуациях. Не продемонстрировал компетенций, которые делали бы его склонным к исполнительской работе именно в предпринимательских проектах, но может быть рекомендован к включению в широкий спектр проектов на позиции инженера или предметного специалиста.',
                 'Участник продемонстрировал базовые позитивные компетенции исполнителя: ответственность и стрессоустойчивость. Это позитивные качества для инженера или предметного специалиста в практически любых проектах. Значительных позитивных характеристик сверх этого участник не продемонстрировал - включая полное отсутствие интереса к позиции технологического предпринимателя. Но участник также и не продемонстрировал негативных характеристик, что делает его кандидатом среднего звена на любую исполнительскую позицию, связанную с его предметным профилем.'
                ],
            'Предприниматель 2':
                ['Участник проявил заметную мотивацию к предпринимательской деятельности и даже успешно выполнил ряд действий по строительству технологической компании, однако не смог осуществить главного предпринимательского действия - продолжить вести деятельность в ситуации неопределённости. Для этого участнику не хватило как уровня базовых предпринимательских компетенций, так и стрессоустойчивости. В результате участник не может быть рекомендован к включению в предпринимательские проекты, так как подобное действие будет сопряжено со слишком высокими финансовыми рисками.',
                 'Активный участник, продемонстрировавший базовые компетенции свойственные технологическим предпринимателям в сфере организации труда, но не обладающий достаточной мотивацией для долгосрочной и последовательной работы в предпринимательской позиции. Подобный недостаток резильентности привёл к тому, что участник не смог продемонстрировать ключевой предпринимательской компетенции - способности самостоятельно определять рамки деятельности компании в ситуации отсутствия предложенного извне курса действий. Не может быть рекомендован к включению в предпринимательские проекты.',
                 'Участник проявил интерес к позиции технологического предпринимателя и опробовал себя в ней, совершив ряд адекватных действий по организации труда в компании, но не смог продемонстрировать уровня компетенций и мотивации, достаточных для того, чтобы быть рекомендованным к включению в предпринимательские проекты. Ключевым дефицитом участника является отсутствие у него способности к самостоятельно конструировать деятельность в отсутствии внешнего заказа на деятельность - ключевой компетенции технологического предпринимателя. Данный дефицит усугубляется недостаточным уровнем резильентности участника, что привело к тому, что он оставил деятельность предпринимателя задолго до получения позитивных результатов. В такой ситуации любая попытка включить участника в предпринимательский проект будет влечь за собой большой финансовый риск, и потому не рекомендована.'
                ],
            'Предприниматель 1':
                ['Участник продемонстрировал базовый интерес к позиции технологического предпринимателя, однако смог включиться в деятельность данной позиции только до момента возникновения в ней потребности к работе с техническим заданием и принятия экономически обоснованных решений. Вероятнее всего это вызвано наличием у участника искажённых и чрезмерно романтизированных представлений о деятельности технологического предпринимателя, от которых он не собирается отказываться. В текущем состоянии не может быть рекомендован к включению в предпринимательские проекты или организационные проекты широкого профиля.',
                 'Участник опробовал себя в позиции предпринимателя, однако, столкнувшись с первыми реалиями деятельности данной позиции, отказался от попыток в неё включиться. Вероятнее всего это связано с наличием у участника искажённых представлений о деятельности технологического предпринимателя как гениального инженера, а не как позиции, отвечающей за конструирование деятельности. При этом участник не продемонстрировал готовности пересматривать это убеждение, и как следствие не может быть рекомендован к включению в предпринимательские проекты.',
                 'Участник продемонстрировал мотивацию к переходу в предпринимательскую позицию, однако данного уровня мотивации не хватило, чтобы заставить участника пересмотреть свои представления о характере деятельности технологического предпринимателя как "ведущего инженера" в группе-инженеров энтузиастов, когда он столкнулся с экономическими реалиями, делающими подобную модель работы невозможной. Глубоко не-экономические представления о предпринимательской деятельности, и отсутствие готовности их пересматривать исключают возможность включения данного участника в предпринимательские проекты.'
                ],
            'Инженер 2':
                ['Участник проявил интерес к исполнительской работе в рамках предпринимательского проекта, но не проявил достаточного уровня стрессоустойчивости для того, чтобы выполнять эту работу значительное количество времени. Без данной базовой компетенции участник не может быть рекомендован к включению в предпринимательские или организационные проекты.',
                 'Участник опробовал себя в роли исполнителя в рамках предпринимательского проекта, однако продемонстрировал нехватку резильентности, необходимой для стабильной работы в предпринимательских проектах. Без данной компетенции не может быть рекомендован к включению в предпринимательские проекты даже на исполнительских позициях.',
                 'Участник не продемонстрировал уровень стрессоустойчивости и резильентности, необходимы для работы в предпринимательских проектах на позиции исполнителя, в которой он себя опробовал. Без данных компетенций и значительного интереса к предпринимательской экономике, который также не был участником проявлен, он не может быть рекомендован к включению в предпринимательские проекты или организационные проекты.'
                ],
            'Инженер 1':
                ['Участник проявил базовый интерес к исполнительской работе в рамках предпринимательской экономики, однако столкнувшись с первыми формами формальной организации труда (такими как трудоустройство и техническое задание) не смог вести деятельность в их рамках. Вероятнее всего это связано с нехваткой мотивации и полным отсутствием опыта прикладной исполнительской деятельности. Участник может быть рекомендован к включению в простые формы организационной деятельности исключительно с целью накопления опыта.',
                 'Участник опробовал себя в позиции инженера в рамках предпринимательского проекта, и, столкнувшись со свойственными реальной предпринимательской экономике требованиями (следование техническому заданию, соблюдение рамок позиции) выпал из деятельности. Вероятнее всего это связано с отсутствием значительной мотивации или искажёнными представлениями об устройстве реального бизнеса. На данном уровне мотивации и подготовки участник не может быть рекомендован к включению в какие-либо проекты.',
                 'Участник не проявил интереса или мотивации к предпринимательскому типу действия, но опробовал себя в позиции инженера-исполнителя в рамках предпринимательского проекта. Участник не смог работать в условиях требований к организации труда, свойственных подобным проектам - следование техническому заданию, соблюдение рамок исполнительской позиции и т.д. Вероятнее всего это связано с отсутствием значительной мотивации или искажёнными представлениями об устройстве реального бизнеса. На данном уровне мотивации и подготовки участник не может быть рекомендован к включению в какие-либо проекты.'
                ],
            'Инженер 0':
                ['Участник не проявил никакого интереса к участию в тренинге или работе в любой из предложенных позиций. Это может быть связано как с нехваткой мотивации к деятельности, так и с отсутствием базовых экономических знаний, информирующих мотивацию к занятию той или иной деятельностной позиции. До появления подобной мотивации участник не может быть рекомендован к включению в какие-либо формы деятельности или связанные с ними позиции.'
                ]
            }

# UI

In [10]:
import warnings
warnings.filterwarnings('ignore')

sg.theme('Reddit')
window = make_window()

class MyHTMLParser(HTMLParser):
    def handle_data(self, data):
        if not re.match(r'^\s*$', data):
            result_list.append(data)

while True:
    event, values = window.read()
    if event in (None, 'Exit', 'Выйти'):
        break
    if event == 'Запустить':
        for folder_make in ['Вывод']:
            try:
                os.mkdir(folder_make)
            except:
                pass
        game_date = datetime.datetime.strptime(values['folder_path'][values['folder_path'][:-5].rfind(' ')+1:-5], '%d%m%y').strftime('%d.%m.%y')
        city_name = values['folder_path'][values['folder_path'].find('/mega')+6:values['folder_path'].find(' ')]
        values['game_date'] = game_date
        values['city_name'] = city_name
        filepath = f'Вывод/ОтчетПКПК_{game_date}_{city_name}.docx'
        ###filepath = f'Вывод/ОтчетПКПК_short_{game_date}_{city_name}.docx'
        folder_path = values['folder_path']+'/'
        comp_path = folder_path+'level_4.html'
        
        for file_name in os.listdir(folder_path):
            new_file_name = file_name.replace('й', 'й').replace('ё', 'ё')
            os.rename(folder_path + file_name, folder_path + new_file_name)
        
        for file_name in os.listdir(folder_path):
            try:
                if file_name.startswith("_игрок") and file_name.endswith(".html"):
                    transactions = pd.read_html(folder_path+file_name,encoding = 'utf-8')[0]
                    result_list = []
                    parser = MyHTMLParser()
                    with io.open(folder_path+file_name, encoding = 'utf-8') as file:
                        for line in file:
                            parser.feed(line)
                    player_name, result_table, result_list = html_read(result_list)
            except:
                print("Удален пустой файл: "+file_name)
                os.remove(folder_path+file_name)
                
        cwd_old = os.getcwd()
        final_table = main(values['folder_path'])
        final_table.to_excel(f'{cwd_old}\\Гипертаблица.xlsx')
        print(f'Файл создан: {cwd_old}\\Гипертаблица.xlsx')
        os.chdir(cwd_old)
                
        last_echelon_comment = ''
        table = table_load()
        trans_dict = counter(values)
        table_final = counter_new(table)
        print('Файл создан: '+filepath)
        
        print('Длина таблицы: '+str(len(table_final)))
        ###
        """
        import shutil
        m1 = folder_path+'generated/graph/new_generate_level2.png'
        m2 = folder_path+'generated/graph/new_generate_level3.png'
        m3 = folder_path+'generated/graph/new_generate_level4.png'
        for index, img_path in enumerate([m1, m2, m3]):
            shutil.copyfile(img_path, f'C:/Users/Андрей/mega/Вывод/Уровень_{index+1}_Дата_тренинга_{game_date}_{city_name}.png')
        """
window.close()

from docxtpl import DocxTemplate, InlineImage, RichText
from html.parser import HTMLParser
from bs4 import BeautifulSoup as BS
from docx.shared import Mm, Cm
import PySimpleGUI as sg
import pandas as pd
import datetime
import os
import io
import re
import regex
import docx
import pandas as pd

tb = pd.read_excel('D:/Старые документы Андрея/Что-то странное с диска С/Downloads/Telegram Desktop/22.10.22.xlsx')

echelon_5 = []
values = {}
df = pd.DataFrame(columns=['№', 'ФИО', 'Почта', 'Телефон', 'Роль', 'Баланс', 'Статус', 'Дата тренинга'])
for record in tb.to_dict(orient='records'):
    item = {'name': record['ФИО участника'].title(), 'email': 'пропуск', 'date': record['Дата тренинга'], 'role': 'Инженер',
            'score': -30, 'rating': record['№'], 'status': 'Выбыл на 0 уровне', 'transactions': 0, 'phone': 'пропуск'}
    item['map'] = 'Участник не совершил достаточно экономических действий для формирования графа деловых связей'
    item['comment'] = 'Участник не проявил никакого интереса к участию в тренинге или работе в любой из предложенных позиций. Это может быть связано как с нехваткой мотивации к деятельности, так и с отсутствием базовых экономических знаний, информирующих мотивацию к занятию той или иной деятельностной позиции. До появления подобной мотивации участник не может быть рекомендован к включению в какие-либо формы деятельности или связынне с ними позиции.'
    item['date'] = datetime.datetime.strptime(str(item['date']), '%Y-%m-%d %H:%M:%S').strftime('%d.%m.%y')
    echelon_5.append(item)
    df.loc[len(df)] = [item['rating'], item['name'], item['email'], item['phone'], item['role'], item['score'], 'Выбывший', item['date']]
    values['echelon_5'], values['echelon_5_len'] = echelon_5, len(echelon_5)
    values['col_labels_final'] = df.columns.values
    values['table_5'] = df.values
    
date = item['date']
doc = DocxTemplate("Ввод_new/wablon.docx")
doc.render(values)
doc.save(f'Вывод/Новый эшелон/{date}_new.docx')
df.to_excel(f'Вывод/Новый эшелон/{date}_new.xlsx', index=False)
df

blacklist = ['Глеб Черепанов', 'Черепанов Глеб', 'Андрей Богдашёв', 'Богдашёв Андрей', 'Андрей Богдашев', 'Богдашев Андрей',\
        ' Михаил Ковалёв', 'Ковалёв Михаил', 'Михаил Ковалев', 'Ковалев Михаил', 'Столярова Полина', 'Полина Столярова',\
        'Кристина Клюева', 'Клюева Кристина', 'Светлана Андреева', 'Андреева Светлана', 'Владислав Богдан', 'Богдан Владислав',\
        'Богдан Влад', 'Латышев Николай', 'Червоная Ольга', 'Александр Кожевников', 'Попов Дмитрий',\
        'Бабенков Виктор', 'Щербич Данила', 'Щербич Данил', 'Тимофеев Андрей', 'Параев Павел', 'Ильина Наташа',\
        'Червоная Ольга', 'Червонная Ольга', 'Червоная Оля', 'Андреева Светлана', 'Андреева Света', 'Клюева Кристина',\
        'Сыромятников Максим', 'Саруханова Диана', 'Лагнер Александра', 'Лагнер Саша', 'Маяренков Сергей', 'Кучеренко Максим',\
        'Карелина Юлия', 'Логов Александр', 'Директор НИИ'
]

def table_load(xxx):
    file_path_table = xxx
    table = pd.read_html(file_path_table,encoding = 'utf-8')[0]
    table = table[table['Фамилия'].apply(lambda x: not x.startswith('Бот') and not x.startswith('Аватар'))]
    table['Баланс'] = table['Баланс'].apply(int)
    table['ФИО'] = table['Фамилия']+' '+table['Имя']+' '+table['Отчество']
    table = table[table['ФИО'].apply(lambda x: all([y not in x for y in blacklist]))]
    table = table[table['Роль'].apply(lambda x: not x.startswith('Директор Фонда'))]
    table = table.reset_index()
    if 'index' in table.columns:
        table.drop('index', axis=1, inplace=True)
    return(table)

import os, pandas as pd
dfs = []
for folder_name in os.listdir()[:]:
    try:
        df = table_load(f'{folder_name}/{folder_name}/data/_сквозной_список_все_личные_счета.html')
        dfs.append(df)
        print(folder_name)
    except:
        pass
    
df_final = pd.concat(dfs)    